In [1]:
import jax.numpy as np
from s5.multi_ssm import discretize_zoh,discretize_bilinear
from flax.training import checkpoints
import os

In [2]:
checkpoint_dir = os.path.abspath("../../cache_dir/checkpoints/")
training_state = checkpoints.restore_checkpoint(checkpoint_dir, target=None,step=0)

/local2/neeraj/miniconda3/envs/icl/lib/python3.10/site-packages/orbax/checkpoint/type_handlers.py:1442: UserWarning: Couldn't find sharding info under RestoreArgs. Populating sharding info from sharding file. Please note restoration time will be slightly increased due to reading from file instead of directly from RestoreArgs. Note also that this option is unsafe when restoring on a different topology than the checkpoint was saved with.
  warnings.warn(


In [3]:
training_state["params"]["layers_0"]["seq"]["w_q"]

Array([[-2.2591485e-03,  8.6951035e-01,  2.1167498e-03],
       [ 5.7084262e-03, -1.5967374e-03,  5.9632155e-05],
       [ 3.7804915e-04, -2.9801710e-03,  1.2836921e-03]], dtype=float32)

In [4]:
training_state["params"]["layers_0"]["seq"]["D"]

Array([2.0865197e-03, 3.8355046e-03, 2.2160461e+00], dtype=float32)

In [5]:
discretization = "zoh"
step_rescale = 1.0
lambda_bar_all = []
Lambda_re = training_state["params"]["layers_0"]["seq"]["Lambda_re"]
log_step = training_state["params"]["layers_0"]["seq"]["log_step"]
for param_id in range(Lambda_re.shape[0]):
            step = step_rescale * np.exp(log_step[:, 0])
            # Discretize
            if discretization in ["zoh"]:
                Lambda_bar, _ = discretize_zoh(Lambda_re[param_id,:], training_state["params"]["layers_0"]["seq"]["B"], step)
            else:
                raise NotImplementedError("Discretization method {} not implemented".format(discretization))
            lambda_bar_all.append(Lambda_bar)
Lambda_bar = np.vstack(lambda_bar_all)

2024-09-05 21:45:17.929215: W external/xla/xla/service/gpu/nvptx_compiler.cc:836] The NVIDIA driver's CUDA version is 12.1 which is older than the PTX compiler version (12.6.20). Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [6]:
Lambda_bar

Array([[0.99797916, 0.9876682 , 0.99804324, 0.99905527, 0.9826968 ,
        0.99757224, 0.99798065, 0.981597  , 0.99860203, 0.99587005],
       [0.9980165 , 0.9881562 , 0.9979048 , 0.9990576 , 0.9825452 ,
        0.9975399 , 0.9978639 , 0.98184836, 0.99849004, 0.99631315],
       [0.9980955 , 0.98801816, 0.99777615, 0.999086  , 0.9819617 ,
        0.99737644, 0.99802953, 0.97979414, 0.9983882 , 0.9959565 ],
       [0.9979366 , 0.98819125, 0.99815917, 0.9989699 , 0.98266155,
        0.9972313 , 0.9978717 , 0.9792623 , 0.9984756 , 0.9963199 ],
       [0.9977545 , 0.98721784, 0.99797875, 0.9990745 , 0.9822484 ,
        0.9972991 , 0.99782455, 0.9805741 , 0.9984486 , 0.99622375],
       [0.99812174, 0.9876904 , 0.9978059 , 0.9990979 , 0.98455966,
        0.99748665, 0.9980299 , 0.98067003, 0.9986708 , 0.99590605],
       [0.99829   , 0.98844403, 0.9978629 , 0.9990154 , 0.98342466,
        0.99741024, 0.9978635 , 0.9814823 , 0.99846894, 0.99560684],
       [0.99788547, 0.98809797, 0.9980569

In [7]:
training_state["params"]["layers_0"]["seq"]["C"]

Array([[ 0.5403625 , -0.03453678, -0.16147475, -0.08379882, -0.0069548 ,
        -0.15041003, -0.00314249, -0.08306681, -0.10927038, -0.06145352],
       [-0.06933011,  0.61434007, -0.14597163, -0.07338666, -0.00932063,
        -0.13796572, -0.01619328, -0.08774588, -0.11519384, -0.06326514],
       [-0.06480269, -0.02266749,  0.46160737, -0.07544587, -0.00233121,
        -0.14172402, -0.01880251, -0.07824241, -0.10310219, -0.05823515],
       [-0.06668413, -0.03445091, -0.16091336,  0.53645426, -0.01061428,
        -0.14593303, -0.01119819, -0.08739482, -0.12035921, -0.06495596],
       [-0.06820043, -0.03556534, -0.14917643, -0.07812142,  0.64899516,
        -0.14757723, -0.01444324, -0.09107143, -0.10692279, -0.06432994],
       [-0.06317776, -0.03180595, -0.14587156, -0.07309453, -0.01355932,
         0.46327367, -0.0130528 , -0.07955518, -0.11615185, -0.05370225],
       [-0.06026293, -0.03032828, -0.1520356 , -0.07779902, -0.02455281,
        -0.15041474,  0.59458154, -0.09313975